<a href="https://colab.research.google.com/github/xanasa14/MLImplementations/blob/master/NetflixVisualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
import re
nf=pd.read_csv("/content/drive/MyDrive/Netflix/netflix_titles.csv")
nf.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [30]:
nf.director.fillna("Unknown",inplace=True)
nf.dropna(inplace=True)
#nf.drop('show_id',axis=1)
nf.drop(columns=['show_id'])
nf.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 6643 entries, 0 to 7785
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       6643 non-null   object
 1   type          6643 non-null   object
 2   title         6643 non-null   object
 3   director      6643 non-null   object
 4   cast          6643 non-null   object
 5   country       6643 non-null   object
 6   date_added    6643 non-null   object
 7   release_year  6643 non-null   int64 
 8   rating        6643 non-null   object
 9   duration      6643 non-null   object
 10  listed_in     6643 non-null   object
 11  description   6643 non-null   object
dtypes: int64(1), object(11)
memory usage: 674.7+ KB


In [31]:
nf.country=nf.country.apply(lambda x:x.split(",")).apply(lambda x:x[0])
nf.date_added=nf.date_added.apply(lambda x:dt.strptime(x.strip(),"%B %d, %Y"))
nf['year_added']=nf.date_added.dt.year
nf['month_added']=nf.date_added.dt.month
nf['day_added']=nf.date_added.dt.day

In [32]:
nf_movie=nf.loc[nf.type=="Movie",:]
nf_tv=nf.loc[nf.type=="TV Show",:]

In [33]:
nf.loc[nf.rating.isin(["TV-MA","R","NC-17"]),"rating"]="Adult"
nf.loc[nf.rating.isin(["TV-14","PG-13"]),"rating"]="Teen"
nf.loc[~nf.rating.isin(["Adult","Teen"]),"rating"]="All Ages"